## Thu thập dữ liệu

`MASHVISOR` cung cấp các API trả về các thông tin liên quan đến bất động sản. [Link API document](https://www.mashvisor.com/api-doc/#the-property-object) \
Nhóm tập trung lấy thông tin các căn nhà hiện đang được bán ở thành phố Chicago bang IL, nước Mỹ.

### Import

In [1]:
import requests
import pandas as pd
import csv
import time

### Gọi API

API được dùng: https://api.mashvisor.com/v1.1/client/property?id=2214791&state=IL \
Gồm 2 tham số bắt buộc: 
- id: mã số của ngôi nhà
- state: tên bang 

In [3]:
# Đọc mã id đã lấy được
f = open("id.txt", "r")
crawlID = f.read().split("\n")
f.close()

In [11]:
# Lấy 100 id để gọi api
id_s = crawlID[1165:1170]

In [12]:
method = "get"
headers = {
    'x-api-key': "3204a295-8a89-4545-96de-4499e3acc430" # Người dùng cung cấp
    }
jsonResult= []

for id_ in id_s:
    url = f'https://api.mashvisor.com/v1.1/client/property?id={int(id_)}&state=IL'
    rsp = requests.request(method, url, headers=headers)
    response = rsp.json()
    if response['status'] == 'success':
        jsonResult.append(response["content"])

    print(response['status'], id_)
    time.sleep(2)

success 2307994
success 2486379
success 2529903
success 2486216
success 2520955


In [ ]:
# Ghi dữ liệu
f=open('data_raw.csv', 'a')
f.write(pd.DataFrame.from_dict(jsonResult).to_csv(index=False, header=None))
f.close()

## Khám phá + tiền xử lí (lựa chọn cột có thể sử dụng cho bài toán)

### Đọc dữ liệu từ file vào DataFrame

In [8]:
df = pd.read_csv('data_raw.csv')
df.head(1)

,isShortSale,source,yearBuilt,nextOpenHouseEndTime,sqft,lastSaleDate,id,state,county,longitude,...,hoa_dues,view_type,parcel_number,architecture_style,has_pool,is_water_front,needs_repair,tenant_occupied,is_market_place,schools
0,NaN,Chicagoland Brokers Inc.,1955.0,NaN,1100.0,NaN,2521392,IL,Cook County,-87.720482,...,NaN,NaN,1.935310e+13,Other,NaN,NaN,0,0,0,"[{'category': 'Elementary', 'name': None, 'dis..."


### Khám phá dữ liệu

#### Số dòng, cột

In [9]:
df.shape

(1165, 61)

#### Kiểm tra số dòng bị lặp lại

In [10]:
df.duplicated().sum()

0

#### Các cột của dữ liệu

In [11]:
df.columns

Index(['isShortSale', 'source', 'yearBuilt', 'nextOpenHouseEndTime', 'sqft',
       'lastSaleDate', 'id', 'state', 'county', 'longitude', 'zip', 'tax',
       'mls_id', 'daysOnMarket', 'homeType', 'property_type',
       'property_sub_type', 'beds', 'num_of_units', 'favorite', 'city',
       'saleType', 'latitude', 'nextOpenHouseDate', 'recentReductionDate',
       'title', 'rent_appreciation_rate', 'originalListPrice', 'parkingSpots',
       'parkingType', 'address', 'nextOpenHouseStartTime', 'lotSize', 'url',
       'baths', 'address_revealing', 'location', 'interested', 'listPrice',
       'price_per_sqft', 'lastSalePrice', 'is_foreclosure',
       'foreclosure_status', 'occupancy_status', 'owner_occupied',
       'heating_system', 'cooling_system', 'walkscore',
       'investment_likelihood_label', 'investment_likelihood_score',
       'investment_likelihood_stars', 'hoa_dues', 'view_type', 'parcel_number',
       'architecture_style', 'has_pool', 'is_water_front', 'needs_repair',


#### Ý nghĩa của các cột:
- `isShortSale`: nhà đó có phải dạng cho thuê ngắn hạn hay không
- `source`:  tên tổ chức cho phép này được mua bán
- `yearBuilt`: năm xây dựng căn nhà
- `nextOpenHouseEndTime`: thời gian mở bán kết thúc
- `sqft`: diện tích 
- `lastSaleDate`: ngày bán cuối cùng
- `id`: mã nhà
- `state`: tiểu bang
- `county`: quận
- `longitude`: kinh độ
- `zip`: mã bưu điện 
- `tax`: tiền thuế
- `mls_id` : mã MLS
- `daysOnMarket`: số ngày kể từ khi được đăng bán
- `homeType`: thuộc tính phụ do nhà cung cấp MLS cung cấp
- `property_type`: thuộc tính chính do nhà cung cấp MLS cung cấp
- `property_sub_type`: danh mục tài sản bất đông sản
- `beds`: số phòng ngủ 
- `num_of_units`: số lượng nhà trong trường hợp bất động sản đa hệ gia đình (multifamily properties)
- `favorite` : được yêu thích hay không
- `city`: thành phố
- `saleType`: nhà cung cấp
- `latitude`: vĩ độ
- `nextOpenHouseDate`: thời gian khai trương mở bán
- `recentReductionDate`: ngày giảm giá gần đây
- `title`:  tiêu đề
- `rent_appreciation_rate`:  tỉ lệ thuê/mua
- `originalListPrice`:  số tiền mở bán lúc mới bắt đầu
- `parkingSpots`: số lượng bãi giữ xe
- `parkingType`: loại bãi giữ xe
- `address`: địa chỉ 
- `nextOpenHouseStartTime`: thời gian mở bán
- `lotSize`: diện tích sử dụng 
- `url`: URL
- `baths`: số lượng phòng tắm 
- `address_revealing`: cho phép lộ địa chỉ
- `location`: khu vực lân cận 
- `interested`: hứng thú
- `listPrice`: giá căn nhà
- `price_per_sqft`: giá trên 1 đơn vị diện tích 
- `lastSalePrice`: giá bán cuối cùng 
- `is_foreclosure`: tịch thu hay không
- `foreclosure_status`:  tình trạng tịch thu nhà
- `occupancy_status`: tình trạng tài sản được sử dụng
- `owner_occupied`: sở hữu bởi chủ nhà
- `heating_system`: hệ thống sưởi
- `cooling_system`: hệ thống làm lạnh 
- `walkscore`: giá trị điểm số
- `investment_likelihood_label`: nhãn thể hiện khả năng đầu tư
- `investment_likelihood_score`: số điểm thể hiện khả năng đầu tư
- `investment_likelihood_stars`: số sao thể hiện khả năng đầu tư
- `view_type`: view căn nhà 
- `parcel_number`: APN tài sản được chỉ định bởi người định thuế
- `architecture_style`: loại kiến trúc  
- `has_pool`: có hồ bơi
- `is_water_front`: nhà hướng sông
- `needs_repair`: có cần chuẩn bị gì không
- `tenant_occupied`: sở hữu bởi người thuê nhà 
- `is_market_place`: gần chợ hay không
- `schools`: trường học

Có thể thấy là có quá nhiều cột trong dữ liệu. Để có thể làm bài toán của nhóm. Tụi em quyết định chỉ chọn ra các cột thật sự cần thiết trong việc dự đoán giá nhà ở như `beds`, `baths`, `sqft`, `property_type`, `parkingSpots`,`lotSize`, `listPrice`, `heating_system`, `cooling_system`, `view_type`, `architecture_style`, `has_pool`, `needs_repair`.

In [41]:
new_df = df[['sqft', 'homeType', 'beds', 'parkingSpots','lotSize', 'baths','listPrice', 
             'heating_system', 'cooling_system', 'view_type', 'architecture_style', 'needs_repair', 'has_pool']]
new_df.head()

,sqft,homeType,beds,parkingSpots,lotSize,baths,listPrice,heating_system,cooling_system,view_type,architecture_style,needs_repair,has_pool
0,1100.0,Single Family Residential,3,NaN,3781.0,2,235000,Forced Air (Natural Gas),Central A/C,NaN,Other,0,NaN
1,2200.0,Single Family Residential,4,2.0,4234.0,3,260000,Forced Air (Natural Gas),Central A/C,NaN,Bungalow,0,NaN
2,750.0,Condo/Coop,2,NaN,NaN,1,275000,Forced Air (Natural Gas),Central A/C,NaN,Other,0,NaN
3,1248.0,Single Family Residential,3,2.0,5279.0,2,309900,NaN,Central A/C,NaN,Ranch,0,NaN
4,1337.0,Single Family Residential,3,2.0,NaN,2,324900,Forced Air (Natural Gas),Central A/C,NaN,Georgian,0,NaN


#### Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

In [42]:
new_df.dtypes

sqft                  float64
homeType               object
beds                    int64
parkingSpots          float64
lotSize               float64
baths                   int64
listPrice               int64
heating_system         object
cooling_system         object
view_type             float64
architecture_style     object
needs_repair            int64
has_pool               object
dtype: object

In [43]:
pd.set_option('display.max_colwidth', 200)
def missing_percentage(c):
    return (c.isna().mean() * 100).round(1)
def num_values(df):
    return df.nunique()
def value_percentages(c):
    return dict((c.value_counts(normalize=True) * 100).round(1))
new_df.agg([missing_percentage, num_values, value_percentages])

,sqft,homeType,beds,parkingSpots,lotSize,baths,listPrice,heating_system,cooling_system,view_type,architecture_style,needs_repair,has_pool
missing_percentage,20.5,0,0,20.3,43.4,0,0,29.8,1,100,0,0,97.5
num_values,458,4,5,10,278,5,474,21,5,0,16,1,1
value_percentages,"{1200.0: 3.9, 1000.0: 3.3, 1100.0: 2.4, 1500.0: 2.4, 1300.0: 2.3, 900.0: 2.3, 1400.0: 1.9, 1600.0: 1.9, 800.0: 1.6, 2000.0: 1.6, 750.0: 1.3, 1800.0: 1.3, 2200.0: 1.2, 2400.0: 1.2, 3000.0: 1.1, 850...","{'Single Family Residential': 62.4, 'Condo/Coop': 37.1, 'Townhouse': 0.4, 'Other': 0.1}","{3: 31.3, 4: 31.2, 2: 22.9, 1: 12.3, 0: 2.2}","{2.0: 58.0, 1.0: 37.5, 3.0: 2.4, 4.0: 1.1, 6.0: 0.3, 100.0: 0.3, 98.0: 0.1, 50.0: 0.1, 0.0: 0.1, 5.0: 0.1}","{3746.0: 5.0, 3123.0: 5.0, 3136.0: 5.0, 3751.0: 4.7, 3781.0: 2.9, 3790.0: 2.7, 3703.0: 2.0, 3101.0: 1.7, 3149.0: 1.7, 3006.0: 1.5, 3720.0: 1.2, 5001.0: 1.2, 4008.0: 1.2, 4356.0: 1.1, 3093.0: 1.1, ...","{2: 47.3, 1: 32.2, 3: 14.7, 4: 5.2, 5: 0.7}","{275000: 1.5, 299900: 1.5, 270000: 1.0, 175000: 1.0, 325000: 1.0, 279900: 0.9, 215000: 0.9, 220000: 0.9, 195000: 0.9, 239900: 0.9, 199000: 0.9, 250000: 0.9, 309900: 0.8, 189000: 0.8, 230000: 0.8, ...","{'Forced Air (Natural Gas)': 59.3, 'Fireplace (Natural Gas)': 9.9, 'Forced Air': 7.0, 'Hot Water': 4.0, 'Baseboard (Natural Gas)': 3.2, 'Baseboard': 3.1, 'Radiator': 2.9, 'Hot Water (Natural Gas)'...","{'Central A/C': 72.8, 'None': 14.1, 'Window Unit(s) A/C': 12.7, 'Zoned A/C': 0.3, 'Partial': 0.1}",{},"{'Other': 62.7, 'Bungalow': 9.3, 'Ranch': 8.2, 'High Rise': 6.0, 'Cape Cod': 3.0, 'Loft': 2.1, 'Georgian': 2.0, 'Split Level': 1.5, 'Contemporary': 1.3, 'English': 0.9, 'Cottage': 0.9, 'New Tradit...",{0: 100.0},{True: 100.0}


### Tiền xử lý